In [ ]:
import numba
import numpy as np

a = np.random.rand(16 * 100_000)
b = np.random.rand(16 * 100_000)

# 1. Numba: `parallel=True` a `prange`

Numba umí část smyček zparalelizovat automaticky. Největší kontrolu ale máme tehdy, když explicitně použijeme `prange` místo `range`.

## 1.1 Co je důležité vědět předem

Numba používá pro CPU paralelizaci model sdílené paměti (typicky přes OpenMP/TBB podle buildu). V této lekci se soustředíme na CPU.

- Paralelní optimalizace zapneme parametrem `parallel=True`.
- Ne každá NumPy operace se zparalelizuje automaticky; záleží na tom, co Numba umí přeložit.
- Smyčky paralelizujeme přes `numba.prange(...)`.
- Operace typu `acc += ...` v `prange` fungují jako redukce, zatímco přepis proměnné bez redukce může vést k jinému chování.

In [ ]:
@numba.jit(nopython=True, parallel=True)
def my_dot_numba_parallel(a, b):
    result = 0.0
    for i in numba.prange(a.shape[0]):
        result += a[i] * b[i]
    return result

In [ ]:
%timeit c = my_dot_numba_parallel(a, b)

In [ ]:
%timeit c = np.dot(a, b)

In [ ]:
# kompilace proběhne při prvním volání
_ = my_dot_numba_parallel(a, b)

In [ ]:
from numba import jit, prange

@jit(nopython=True, parallel=True)
def test_prange_nested(x):
    n = x.shape[0]
    sin_x = np.sin(x)
    cos_x = np.cos(sin_x * sin_x)
    acc = 0.0
    for i in prange(n - 2):
        for j in prange(n - 1):
            acc += cos_x[i] + cos_x[j + 1]
    return acc

In [ ]:
test_prange_nested(np.arange(10))

## 1.2 Diagnostika paralelizace

Numba umí ukázat, které části funkce skutečně zparalelizovala.

In [ ]:
test_prange_nested.parallel_diagnostics(level=4)

## 1.3 Norma vektoru z minulé lekce

In [ ]:
import math

@numba.jit(nopython=True, parallel=False, fastmath=True)
def my_norm_numba_serial(a):
    result = 0.0
    for i in range(len(a)):
        result += a[i] ** 2
    return math.sqrt(result)

In [ ]:
x = np.random.rand(4_000_000)
y1 = my_norm_numba_serial(x)
y2 = np.linalg.norm(x)
print(y1, y2)

In [ ]:
%timeit _ = my_norm_numba_serial(x)

In [ ]:
%timeit _ = np.linalg.norm(x)

## 1.4 Paralelní verze normy

In [ ]:
@numba.jit(nopython=True, parallel=True, fastmath=True)
def my_norm_numba_parallel(a):
    result = 0.0
    for i in numba.prange(len(a)):
        result += a[i] ** 2
    return math.sqrt(result)

_ = my_norm_numba_parallel(x)

In [ ]:
%timeit _ = my_norm_numba_parallel(x)

## 1.5 Počet vláken

Aktuální počet vláken zjistíme přes `numba.get_num_threads()`.

In [ ]:
numba.get_num_threads()

Počet vláken můžeme upravit (`set_num_threads`), ale nejvýš na počet logických jader dostupných procesu.

In [ ]:
numba.set_num_threads(2)

In [ ]:
%timeit _ = my_norm_numba_parallel(x)

In [ ]:
@numba.jit(nopython=True, parallel=True)
def my_dot_numba_chunked(a, b):
    result = 0.0
    with numba.parallel_chunksize(1_000_000):
        for i in numba.prange(a.shape[0]):
            result += a[i] * b[i]
    return result

_ = my_dot_numba_chunked(a, b)

In [ ]:
%timeit c = my_dot_numba_chunked(a, b)